### Load Colab Notebook settings

**Link Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Try GPU power**

In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [12]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

Gen RAM Free: 12.4 GB  | Proc size: 1.4 GB
GPU RAM Free: 11372MB | Used: 69MB | Util   1% | Total 11441MB


In [ ]:
#!kill -9 -1

## Train the model

In [ ]:
# mettre dans un .py qaund on a trouvé les bons params

import numpy as np
import sys
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
from keras.preprocessing.image import img_to_array
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as keras

def data_load(directory_name): 
    filenames = os.listdir(directory_name)
    imgs = []
    for i, fileNb in enumerate(filenames):
        full_name = directory_name+fileNb
        img=mpimg.imread(full_name)
        imgr = img_to_array(img)
        imgs.append(imgr)
        sys.stdout.write("\rImage {}/{} is being loaded".format(i+1,len(filenames)))
        sys.stdout.flush()    
    return np.asarray(imgs)

def dataGenerator(batch_size, data, labels, aug_dict, image_save_prefix ="image", mask_save_prefix="mask", seed=1):
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)

    image_generator = image_datagen.flow(
        data,
        batch_size = batch_size,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow(
        labels,
        batch_size = batch_size,
        save_prefix  = mask_save_prefix,
        seed = seed)
    
    for (img,mask) in zip(image_generator, mask_generator):
        if(np.max(img) > 1):
          img = img / 255
          mask = mask /255
          mask[mask > 0.5] = 1
          mask[mask <= 0.5] = 0
        yield (img,mask)

def uneti(input_size, pretrained_weights=None, verbose=True):
    ##Parameters
    model = Sequential()
    kernel_size = (3,3)
    pool_size = (2,2)
    alpha_relu = 0.1
    regularizer = 1e-6

    #Size of input matrix
    #To change according to the shape
    model = Sequential()


    #Add convolution 
    model.add(Convolution2D(64,
                            kernel_size,
                            padding='same',
                            input_shape=input_size))
    model.add(LeakyReLU(alpha_relu))
    model.add(MaxPooling2D(pool_size))
    model.add(Convolution2D(64,
                            kernel_size,
                            padding='same',
                            input_shape=input_size))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha_relu))
    model.add(MaxPooling2D(pool_size))

    model.add(Convolution2D(128,
                            kernel_size,
                            padding='same',
                            input_shape=input_size))
    model.add(LeakyReLU(alpha_relu))
    model.add(MaxPooling2D(pool_size))
    model.add(Convolution2D(128,
                            kernel_size,
                            padding='same',
                            input_shape=input_size))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha_relu))
    model.add(MaxPooling2D(pool_size))

    model.add(Convolution2D(256,
                            kernel_size,
                            padding='same',
                            input_shape=input_size))
    model.add(LeakyReLU(alpha_relu))
    model.add(MaxPooling2D(pool_size))
    model.add(Convolution2D(256,
                            kernel_size,
                            padding='same',
                            input_shape=input_size))
    model.add(Dropout(0.1))
    model.add(LeakyReLU(alpha_relu))

    model.add(Flatten())

    model.add(Dense(2))
    model.add(Activation('softmax'))
    
    
    adam_optimizer = Adam(lr=0.0005)
    model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    if(verbose == True):
        model.summary()

    if(pretrained_weights):
    	  model.load_weights(pretrained_weights)

    return model

**1. Load the data and split it into validation and training**

In [15]:
imgs = data_load(root_path+'data_rs/train/images/')
gts = data_load(root_path+'data_rs/train/groundtruth/')

TEST_SIZE = 0.2
X_train, X_test, gt_train, gt_test = train_test_split(imgs, gts, test_size=TEST_SIZE, random_state=1)

Image 100/100 is being loaded

**2. Data augmentation and generators**

In [ ]:
data_gen_args = dict(rotation_range=0.15,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    shear_range=0.1,
                    zoom_range=0.1,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='nearest')



train_generator = dataGenerator(32, X_train, gt_train, data_gen_args)
validation_generator = dataGenerator(32, X_test, gt_test, data_gen_args)


## LE PROBLÈME EST DANS LA FACON DE GÉNÉNER DES IMAGES: EUX ILS UTILISENT
## CE QUI EST DANS LA CELLULE DU BAS

# ca c'est juste le code utilisé dans notre version dans dataGenerator
image_generator = image_datagen.flow(
        data,
        batch_size = batch_size,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow(
        labels,
        batch_size = batch_size,
        save_prefix  = mask_save_prefix,
        seed = seed)

In [ ]:
#utilisation de la fonction
train_generator = image_generator(self.train_data_split,
                                          self.train_label_split,
                                          self.WINDOW_SIZE,
                                          batch_size = 32, 
                                          upsample=True)
        
validation_generator = image_generator(self.validation_data_split,
                                          self.validation_label_split,
                                          self.WINDOW_SIZE,
                                          batch_size = 32,
                                          upsample=True)


# déclaration de la fonction
def image_generator(images, ground_truths, window_size, batch_size = 64,
                    upsample=False):
    np.random.seed(0)
    imgWidth = images[0].shape[0]
    imgHeight = images[0].shape[1]
    half_patch = constants.IMG_PATCH_SIZE // 2
    
    padSize = (window_size - constants.IMG_PATCH_SIZE) // 2 #42
    paddedImages = []
    for image in images:
        paddedImages.append(pad_image(image,padSize))
        
    while True:
        batch_input = []
        batch_output = [] 
        
        #rotates the whole batch for better performance
        randomIndex = np.random.randint(0, len(images))  
        img = paddedImages[randomIndex]
        gt = ground_truths[randomIndex]
        
        # rotate with probability 10 / 100
        random_rotation = 0
        if (np.random.randint(0, 100) < 10):
            rotations = [90, 180, 270, 45, 135, 225, 315]
            random_rotation = np.random.randint(0, 7)
            img = rot(img, np.array([imgWidth+2*padSize, imgHeight+2*padSize]), rotations[random_rotation])
            gt = rot(gt, np.array([imgWidth, imgHeight]), rotations[random_rotation]) 
        
        background_count = 0
        road_count = 0
        while len(batch_input) < batch_size:
            x = np.empty((window_size, window_size, 3))
            y = np.empty((window_size, window_size, 3))
            
            
            # we need to limit possible centers to avoid having a window in an interpolated part of the image
            # we limit ourselves to a square of width 1/sqrt(2) smaller
            if(random_rotation > 2):
                boundary = int((imgWidth - imgWidth / np.sqrt(2)) / 2)
            else:
                boundary = 0
            center_x = np.random.randint(half_patch + boundary, imgWidth  - half_patch - boundary)
            center_y = np.random.randint(half_patch + boundary, imgHeight - half_patch - boundary)
            
            x = img[center_x - half_patch:center_x + half_patch + 2 * padSize,
                    center_y  - half_patch:center_y + half_patch + 2 * padSize]
            y = gt[center_x - half_patch : center_x + half_patch,
                   center_y - half_patch : center_y + half_patch]
            
            # vertical
            if(np.random.randint(0, 2)):
                x = np.flipud(x)
            
            # horizontal
            if(np.random.randint(0, 2)):
                x = np.fliplr(x)
            
            label = [0., 1.] if (np.array([np.mean(y)]) >  constants.FOREGROUND_THRESHOLD) else [1., 0.]
            
            # makes sure we have an even distribution of road and non road if we oversample
            if not upsample:
                batch_input.append(x)
                batch_output.append(label)
            elif label == [1.,0.]:
                # case background
                background_count += 1
                if background_count != batch_size // 2:
                    batch_input.append(x)
                    batch_output.append(label)
            elif label == [0.,1.]:
                # case road
                road_count += 1
                if road_count != batch_size // 2:
                    batch_input.append(x)
                    batch_output.append(label)
                
        batch_x = np.array( batch_input )
        batch_y = np.array( batch_output )

        yield( batch_x, batch_y )   

i = 0
for (img,mask) in myGene: 
    i = i+1
    print(i, img.shape, mask.shape)

**3. Load the model**

In [17]:
model = uneti(input_size = (100,100,3), pretrained_weights=None, verbose=True)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 100, 100, 64)      1792      
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 100, 100, 64)      0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 50, 50, 64)        36928     
_________________________________________________________________
dropout_7 (Dropout)          (None, 50, 50, 64)        0         
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 50, 50, 64)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 25, 25, 64)       

**4. Train the model**

In [18]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=700,
                    epochs=30, 
                    verbose=1,
                    validation_data = validation_generator,
                    validation_steps = 300)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/30


ValueError: ignored

### Save model to disk

In [ ]:
# serialize model to JSON
OUTPUT_FILENAME = "Model_UNET_400"

model_json = model.to_json()
with open(OUTPUT_FILENAME + ".json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights(OUTPUT_FILENAME + ".h5")
print("Saved model to disk")

1
2
Saved model to disk


### Predictions (don't run it on Colab)

**Load model** [Optional]

In [ ]:
from keras.models import model_from_json
json_file = open('simple_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

**Resize images in a folder** [Optional]

In [ ]:
from PIL import Image
path1 = "##UNET/data/membrane/tmp/"
path2 = "##UNET/data/membrane/test/"

filenames = os.listdir(path1)
for i, fileNb in enumerate(filenames):
    if(fileNb!='.DS_Store'):        
        im1 = Image.open(path1+fileNb)
        side = 400
        # use one of these filter options to resize the image
        im2 = im1.resize((side, side), Image.NEAREST)
        im2.save(path2+fileNb)

In [ ]:
import skimage.io as io
import skimage.transform as trans


def testGeneratorNEW(batch_size, train_path, image_folder, aug_dict, image_save_prefix ="prediction",
                     num_class=2, save_to_dir=None, target_size=(256,256), seed=1):
    image_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    for img in image_generator:
        img = img / 255
        yield img

In [ ]:
def model_UNETOP(input_size = (256,256,3, 1), pretrained_weights=None):
    inputs = Input(shape=(256,256,3))

    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(3, 1, activation = 'sigmoid')(conv9)

    

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model


In [ ]:
from Model_UNET import *

data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
test_generatorNEW = testGeneratorNEW(2, 'UNET/data/membrane','test',data_gen_args,save_to_dir = None)

#testGene = testGenerator("UNET/data/membrane/test")


#test_datagen = ImageDataGenerator(rescale=1./255)
#
#testGene = test_datagen.flow_from_directory(
#        "UNET/data/membrane/test/",
#        target_size=(400, 400),
#        color_mode="rgb",
#        shuffle = False,
#        class_mode='categorical',
#        batch_size=1)

print('1')

model = model_UNETOP()
print('2')

model.load_weights("Model_UNET.h5")
print('3')

results = model.predict_generator(test_generatorNEW, 30,verbose=1)
print('4')

In [ ]:
def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

In [ ]:
saveResult("UNET/data/membrane/test",results)